In [4]:
import torch
import numpy as np
from torch.utils.data import Dataset
import scanpy as sc
import scvelo as scv
import pandas as pd
from torch_geometric.nn import GATConv
import torch.nn.functional as F
from torch import nn

import model, data

In [12]:
scrna_path = "DLPFC_151673\\scrna.h5ad"
st_path = "DLPFC_151673\\filtered_feature_bc_matrix.h5"

scrna = scv.read(scrna_path)

In [13]:
scrna.obs

,cell_type
Ex_10_L2_4.3_Control_B3_AAACCTGAGGTAGCCA,Ex_10_L2_4
Inhib_5.3_Control_B3_AAACCTGCAAACTGTC,Inhib_5
Ex_10_L2_4.3_Control_B3_AAACCTGCAACAACCT,Ex_10_L2_4
Ex_8_L5_6.3_Control_B3_AAACCTGGTCCGAACC,Ex_8_L5_6
Inhib_2_VIP.3_Control_B3_AAACCTGGTCGTGGCT,Inhib_2_VIP
...,...
OPCs_1.17_Suicide_B1_TTTGTCAAGCTCCTTC,OPCs_1
Ex_10_L2_4.17_Suicide_B1_TTTGTCAAGGCTAGCA,Ex_10_L2_4
Ex_10_L2_4.17_Suicide_B1_TTTGTCAAGGGATGGG,Ex_10_L2_4
Inhib_2_VIP.17_Suicide_B1_TTTGTCACATGTCCTC,Inhib_2_VIP


In [10]:
data = data.SingleCellDataset(scrna_path)

Filtered out 5239 genes that are detected 20 counts (shared).
Extracted 2000 highly variable genes.
Logarithmized X.


c:\Users\Tian\AppData\Local\Programs\Python\Python311\Lib\site-packages\scvelo\preprocessing\utils.py:705: DeprecationWarning: `log1p` is deprecated since scVelo v0.3.0 and will be removed in a future version. Please use `log1p` from `scanpy.pp` instead.
  log1p(adata)


computing neighbors
    finished (0:00:00) --> added 
    'distances' and 'connectivities', weighted adjacency matrices (adata.obsp)
computing moments based on connectivities
    finished (0:00:00) --> added 
    'Ms' and 'Mu', moments of un/spliced abundances (adata.layers)


KeyError: "None of [Index(['array_row', 'array_col'], dtype='object')] are in the [columns]"

In [4]:
st = scv.read(st_path)

In [5]:
scrna_spliced = scrna.to_df(layer = 'spliced')
st_spliced = st.to_df(layer = 'spliced')

In [8]:
st.obs

# which is xy?

,orig.ident,nCount_Spatial,nFeature_Spatial,percent.mito,Spatial_snn_res.1,seurat_clusters,scanorama_snn_res.1,Cardiomyocytes-1,Cardiomyocytes-2,Immature.myocardial.cells,...,TMSB4X.high.cells,Epi-mesenchymal.cells,Dendritic.cells,Mural.cells,max,celltype_prediction_max,celltype_prediction,celltype_prediction_mode,region,n_counts
AAACAAGTATCTCCCA,D14,13254,2446,19.277199,7,2,2,0.248662,0.002676,0.291200,...,0.023730,0.000000,0,0.000000,0.291200,MT-enriched cardiomyocytes,Immature myocardial cells,Erythrocytes,Atria,10281.0
AAACATTTCCCGGATT,D14,14259,2439,23.893681,7,2,2,0.285085,0.000000,0.369797,...,0.037476,0.000000,0,0.000000,0.369797,Cardiomyocytes-1,Immature myocardial cells,Vascular endothelial cells,Atria,10281.0
AAACCCGAACGAAATC,D14,6800,1856,14.338235,13,6,6,0.005008,0.000000,0.000000,...,0.042678,0.011129,0,0.021135,0.752036,Fibroblast cells,Fibroblast cells,Fibroblast cells,Epicardium,10281.0
AAACCGTTCGTCCAGG,D14,10725,2090,21.212121,1,0,0,0.397038,0.000000,0.289122,...,0.024527,0.000000,0,0.000000,0.397038,Vascular endothelial cells,Cardiomyocytes-1,Cardiomyocytes-1,Compact LV and \ninter-ventricular septum,10281.0
AAACCTAAGCAGCCGG,D14,9931,1980,15.547276,7,2,2,0.016108,0.002674,0.003484,...,0.004331,0.000000,0,0.000000,0.958324,Erythrocytes,Erythrocytes,Erythrocytes,Atria,10281.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTGTTTCCCGAAAG,D14,15129,2849,12.961861,12,8,8,0.000000,0.000000,0.000000,...,0.000000,0.117749,0,0.032358,0.555955,Fibroblast cells,Fibroblast cells,Fibroblast cells,Valves,10281.0
TTGTTAGCAAATTCGA,D14,8408,1781,20.611323,1,0,0,0.588776,0.039379,0.333438,...,0.017630,0.000000,0,0.000000,0.588776,Immature myocardial cells,Cardiomyocytes-1,Cardiomyocytes-1,Compact LV and \ninter-ventricular septum,10281.0
TTGTTCAGTGTGCTAC,D14,12109,2482,13.461062,4,0,0,0.383047,0.000000,0.364800,...,0.231844,0.000000,0,0.000000,0.383047,Immature myocardial cells,Cardiomyocytes-1,Cardiomyocytes-1,Trabecular LV and \nendocardium,10281.0
TTGTTGTGTGTCAAGA,D14,7822,1741,19.803119,1,0,0,0.867228,0.000000,0.097680,...,0.032034,0.000000,0,0.000000,0.867228,Cardiomyocytes-1,Cardiomyocytes-1,Cardiomyocytes-1,Compact LV and \ninter-ventricular septum,10281.0
